<a href="https://colab.research.google.com/github/HuugoAlexandre/Digital-Innovation-One/blob/main/Teste_Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

# Muda o diretório de trabalho para '/content' (onde o colab tem permissão pra gravar)
os.chdir('/content')


In [ ]:
!wget https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip

--2024-12-21 21:47:35--  https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip
Resolving download.microsoft.com (download.microsoft.com)... 23.192.208.155, 2600:1409:12:2b7::317f, 2600:1409:12:299::317f
Connecting to download.microsoft.com (download.microsoft.com)|23.192.208.155|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 824887076 (787M) [application/octet-stream]
Saving to: ‘kagglecatsanddogs_5340.zip’

kagglecatsanddogs_5 100%[===================>] 786.67M   113MB/s    in 7.6s    

2024-12-21 21:47:42 (104 MB/s) - ‘kagglecatsanddogs_5340.zip’ saved [824887076/824887076]



In [ ]:
import zipfile

# Caminho para o arquivo ZIP
zip_file_path = '/content/kagglecatsanddogs_5340.zip'

# Caminho onde o conteúdo será descompactado
extract_path = '/content/PetImages/PetImages'

# Descompactar o arquivo
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Arquivo descompactado com sucesso!")


Arquivo descompactado com sucesso!


In [ ]:
import os

# Listar diretórios e arquivos na pasta principal
dataset_path = "/content/PetImages/PetImages"
print("Conteúdo da pasta:", os.listdir(dataset_path))


Conteúdo da pasta: ['Dog', 'Cat']


In [ ]:
# Verificar a quantidade de arquivos em cada subpasta
cats_dir = os.path.join(dataset_path, "Cat")
dogs_dir = os.path.join(dataset_path, "Dog")

print("Número de imagens em 'Cat':", len(os.listdir(cats_dir)))
print("Número de imagens em 'Dog':", len(os.listdir(dogs_dir)))


Número de imagens em 'Cat': 12501
Número de imagens em 'Dog': 12501


In [ ]:
# Removendo possíveis arquivos corrompidos
from PIL import Image

def remove_corrupted_images(folder_path):
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        try:
            with Image.open(file_path) as img:
                img.verify()  # Verifica se o arquivo é válido
        except (IOError, SyntaxError):
            print("Arquivo corrompido removido:", file_path)
            os.remove(file_path)

# Remover imagens corrompidas de ambas as pastas
remove_corrupted_images(cats_dir)
remove_corrupted_images(dogs_dir)


Arquivo corrompido removido: /content/PetImages/PetImages/Cat/666.jpg
Arquivo corrompido removido: /content/PetImages/PetImages/Cat/Thumbs.db
Arquivo corrompido removido: /content/PetImages/PetImages/Dog/Thumbs.db


/usr/local/lib/python3.10/dist-packages/PIL/TiffImagePlugin.py:935: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Arquivo corrompido removido: /content/PetImages/PetImages/Dog/11702.jpg


In [ ]:
import shutil
from sklearn.model_selection import train_test_split

# Criar diretórios para treino e validação
base_dir = "/content/cats_dogs"
train_dir = os.path.join(base_dir, "train")
validation_dir = os.path.join(base_dir, "validation")

os.makedirs(os.path.join(train_dir, "cats"), exist_ok=True)
os.makedirs(os.path.join(train_dir, "dogs"), exist_ok=True)
os.makedirs(os.path.join(validation_dir, "cats"), exist_ok=True)
os.makedirs(os.path.join(validation_dir, "dogs"), exist_ok=True)

# Dividir os dados e copiá-los para os diretórios
def split_data(src_dir, train_dst, val_dst, split_ratio=0.8):
    files = [os.path.join(src_dir, f) for f in os.listdir(src_dir) if f.endswith(('.jpg', '.png'))]
    train_files, val_files = train_test_split(files, test_size=1-split_ratio)
    for file in train_files:
        shutil.copy(file, train_dst)
    for file in val_files:
        shutil.copy(file, val_dst)

# Organizar os dados
split_data(cats_dir, os.path.join(train_dir, "cats"), os.path.join(validation_dir, "cats"))
split_data(dogs_dir, os.path.join(train_dir, "dogs"), os.path.join(validation_dir, "dogs"))


In [ ]:
print("Imagens de treino - gatos:", len(os.listdir(os.path.join(train_dir, "cats"))))
print("Imagens de treino - cachorros:", len(os.listdir(os.path.join(train_dir, "dogs"))))
print("Imagens de validação - gatos:", len(os.listdir(os.path.join(validation_dir, "cats"))))
print("Imagens de validação - cachorros:", len(os.listdir(os.path.join(validation_dir, "dogs"))))


Imagens de treino - gatos: 9999
Imagens de treino - cachorros: 9999
Imagens de validação - gatos: 2500
Imagens de validação - cachorros: 2500


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Criar geradores de treino e validação
train_datagen = ImageDataGenerator(rescale=1.0/255)
validation_datagen = ImageDataGenerator(rescale=1.0/255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode="binary"
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode="binary"
)


Found 19998 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [ ]:
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2

# Carregar o modelo MobileNetV2 pré-treinado, excluindo a parte superior (camadas finais)
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

# Congelar as camadas do modelo base para não treinar essas camadas
base_model.trainable = False

# Construir o modelo
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(1, activation='sigmoid')  # Apenas uma saída (gato ou cachorro)
])

# Compilar o modelo
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Resumo do modelo
model.summary()


<ipython-input-14-222e4d8b479b>:5: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(150, 150, 3))


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224 (Functional)    │ (None, 5, 5, 1280)          │       2,257,984 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 1280)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │           1,281 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,259,265 (8.62 MB)

 Trainable params: 1,281 (5.00 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [ ]:
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator
)


Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 491s 784ms/step - accuracy: 0.9790 - loss: 0.0542 - val_accuracy: 0.9628 - val_loss: 0.1043
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 492s 787ms/step - accuracy: 0.9800 - loss: 0.0513 - val_accuracy: 0.9554 - val_loss: 0.1160
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 501s 785ms/step - accuracy: 0.9795 - loss: 0.0550 - val_accuracy: 0.9596 - val_loss: 0.1079
Epoch 4/10
318/625 ━━━━━━━━━━━━━━━━━━━━ 3:16 639ms/step - accuracy: 0.9803 - loss: 0.0558

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

# Caminho para a imagem que você quer testar
img_path = '/content/caminho/para/a/imagem.jpg'

# Carregar a imagem, redimensionando para 150x150 pixels (o tamanho que o modelo espera, mas depende do modelo)
img = image.load_img(img_path, target_size=(150, 150))

# Converter a imagem para um array NumPy e normalizar (escalar os valores de 0 a 1)
img_array = image.img_to_array(img) / 255.0

# Adiciona uma dimensão extra para representar o batch (modelo espera um batch, mesmo que tenha só uma imagem)
img_array = np.expand_dims(img_array, axis=0)

print(img_array.shape)  # Verifica o formato da imagem


In [ ]:
# Fazer a predição com o modelo
prediction = model.predict(img_array)

# A predição será um valor entre 0 e 1, onde valores próximos de 0 indicam "gato" e próximos de 1 indicam "cachorro"
if prediction < 0.5:
    print("A imagem é um Gato!")
else:
    print("A imagem é um Cachorro!")
